In [ ]:
#default_exp service.filesystem

# Service Filesystem

> Service implementation that stores data in the local filesystem.

In [ ]:
#export
import json,time,datetime
from pathlib import Path

In [ ]:
import tempfile, shutil

In [ ]:
#export
def posts_list_to_dict(posts,key='id'):
    "Convert a list of dictionaries to a dictionary of dictionaries"
    return {post[key]:post for post in posts}

In [ ]:
posts=[dict(id=0,tag='a'),dict(id=-1,tag='B')]
expected={0:posts[0],-1:posts[1]}
assert expected==posts_list_to_dict(posts)

In [ ]:
#export
class ServiceFilesystem:
    # TODO: DRY
    def __init__(self,data_dir):
        self.data_dir=Path(data_dir)
        self.data_dir.mkdir(parents=True,exist_ok=True)
        
    def read_user_by_id(self,id): 
        if (self.data_dir/'users.json').is_file():
            with open(self.data_dir/'users.json') as f: 
                for user in json.load(f):
                    if user['id']==id: return user
        return None
    
    def read_user_by_username(self,username): 
        if (self.data_dir/'users.json').is_file():
            with open(self.data_dir/'users.json') as f: 
                for user in json.load(f):
                    if user['username']==username: return user
        return None
    
    def create_user(self,username,password): 
        users=[]
        if (self.data_dir/'users.json').is_file():
            with open(self.data_dir/'users.json') as f: users=json.load(f)
        id=round(time.time()*1000)
        users.append(dict(id=id,username=username,password=password))
        with open(self.data_dir/'users.json','w') as f: json.dump(users,f)
        return id
    
    def _add_username(self,post):
        # TODO: check how slow this is ...
        user=self.read_user_by_id(post['author_id'])
        post['username']='Unknown user' if user is None else user['username'] 
        return post
    
    def read_posts_by_author_id(self,author_id): 
        if (self.data_dir/f'posts-{author_id}.json').is_file():
            with open(self.data_dir/f'posts-{author_id}.json') as f: 
                return [self._add_username(p) for p in json.load(f) if p['is_deleted']==0]
        return []
    
    def read_post_by_id(self,author_id,id): 
        if (self.data_dir/f'posts-{author_id}.json').is_file():
            with open(self.data_dir/f'posts-{author_id}.json') as f:
                for post in json.load(f):
                    if post['id']==id: return self._add_username(post)
        return None
    
    def create_post(self,author_id,title,body):
        posts=[]
        if (self.data_dir/f'posts-{author_id}.json').is_file():
            with open(self.data_dir/f'posts-{author_id}.json') as f: posts=json.load(f)
        id=round(time.time()*1000)
        _now=datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        posts.insert(0,dict(id=id,author_id=author_id,title=title,body=body,created=_now,is_deleted=0))
        with open(self.data_dir/f'posts-{author_id}.json','w') as f: json.dump(posts,f)
        return id
    
    def update_post_by_id(self,author_id,id,title,body):
        if (self.data_dir/f'posts-{author_id}.json').is_file():
            with open(self.data_dir/f'posts-{author_id}.json') as f: posts = json.load(f)
            for post in posts:
                if post['id']==id: 
                    post['title'],post['body']=title,body
                    with open(self.data_dir/f'posts-{author_id}.json','w') as f: json.dump(posts,f)
                    return post
        return None
    
    def delete_post_by_id(self,author_id,id):
        if (self.data_dir/f'posts-{author_id}.json').is_file():
            with open(self.data_dir/f'posts-{author_id}.json') as f: posts = json.load(f)
            for post in posts:
                if post['id']==id: 
                    post['is_deleted']=1
                    with open(self.data_dir/f'posts-{author_id}.json','w') as f: json.dump(posts,f)
                    return post
        return None
    
    def prepare_posts_file_by_author_id(self,author_id):
        if (self.data_dir/f'posts-{author_id}.json').is_file():
            return self.data_dir,f'posts-{author_id}.json'
        return None,None
    
    def upload_posts_from_file(self,author_id,file):
        # TODO: handle non-json formats
        with open(file) as f: 
            posts=json.load(f)
            for post in posts: post['author_id']=author_id
            posts=posts_list_to_dict(posts)
        if (self.data_dir/f'posts-{author_id}.json').is_file():
            with open(self.data_dir/f'posts-{author_id}.json') as f: 
                posts.update(posts_list_to_dict(json.load(f)))
        posts=sorted(posts.values(), key=lambda post: post['id'], reverse=True)
        with open(self.data_dir/f'posts-{author_id}.json','w') as f: json.dump(posts,f)

In [ ]:
temp_path = tempfile.mkdtemp()
try:
    service=ServiceFilesystem(temp_path)
    post=service._add_username(dict(author_id=123))
    assert 'Unknown user'==post['username']
finally:
    shutil.rmtree(temp_path)

In [ ]:
def _compare_post(expected,actal):
    "Check values match for all keys in `expected`, which might not be all keys in `actual`"
    for k in expected.keys(): assert expected[k]==actal[k]

temp_path = tempfile.mkdtemp()
try:
    service=ServiceFilesystem(temp_path)
    # user section
    assert service.read_user_by_id(1234) is None
    assert service.read_user_by_username('test.user') is None
    user_id=service.create_user('test.user','badPassword')
    expected_user=dict(id=user_id,username='test.user',password='badPassword')
    assert service.read_user_by_id(user_id)==expected_user
    assert service.read_user_by_username('test.user')==expected_user
    # post section
    assert service.read_posts_by_author_id(123)==[]
    assert service.read_posts_by_author_id(user_id)==[]
    assert service.read_post_by_id(user_id,123) is None
    for i in range(3): service.create_post(user_id,f'title{i}','body')
    post_id=service.create_post(user_id,'title','body')
    for i in range(3): service.create_post(user_id,f'title{i}2','body')
    # don't add created to `expected_post` as we don't know what it's value will be
    expected_post=dict(id=post_id,author_id=user_id,title='title',body='body',username='test.user',is_deleted=0)
    posts=service.read_posts_by_author_id(user_id)
    assert len(posts)==7
    _compare_post(expected_post,posts[3])
    assert isinstance(posts[3]['created'],str)
    post=service.read_post_by_id(user_id,post_id)
    assert post==posts[3]
    assert post['is_deleted']==0
    assert post!=service.delete_post_by_id(user_id,post_id)
    expected_post['is_deleted']=1
    # deleted posts are readable by ID ...
    _compare_post(expected_post,service.read_post_by_id(user_id,post_id))
    # but are not returned when reading all posts by author
    assert len(service.read_posts_by_author_id(user_id))==6
    # test prep download
    directory,filename=service.prepare_posts_file_by_author_id(user_id)
    assert isinstance(directory,Path)
    assert isinstance(filename,str)
    assert filename==f'posts-{user_id}.json'
    assert (directory/filename).is_file()
    # test upload
    shutil.copyfile(directory/filename,directory/'posts2upload')
    posts=service.read_posts_by_author_id(user_id)
    service.upload_posts_from_file(user_id,directory/'posts2upload') # makes no difference
    assert posts==service.read_posts_by_author_id(user_id)
    [service.delete_post_by_id(user_id,post['id']) for post in posts]
    service.upload_posts_from_file(user_id,directory/'posts2upload')
    assert []==service.read_posts_by_author_id(user_id) # they are all still deleted
    (directory/filename).unlink()
    service.upload_posts_from_file(user_id,directory/'posts2upload')
    assert posts==service.read_posts_by_author_id(user_id)
finally:
    shutil.rmtree(temp_path)

In [ ]:
#export
def before_request(app):
    return ServiceFilesystem(app.config['DATA_DIR'])

In [ ]:
#export
def after_request(app,service):
    pass

In [ ]:
#export
def init_service(app):
    print('service.filesystem.init_service')

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 40a_service_db.ipynb.
Converted 40b_service_filesystem.ipynb.
Converted 50_web_app.ipynb.
Converted 50b_web_auth.ipynb.
Converted 50c_web_blog.ipynb.
Converted index.ipynb.
